# Volume-Based Estimators

This concept was originally developed by <strong data-cite="Lucy1999">[]</strong> and successively refined, for example, by <strong data-cite="Lucy1999a">[]</strong>, <strong data-cite="Lucy2002">[]</strong> and <strong data-cite="Lucy2003">[]</strong>. These estimators use the monte carlo packets to determine how the light-matter interactions in the supernova affect the conditions in the ejecta.

When TARDIS runs, we enter into a loop with two main components: a Monte Carlo iteration occurs, and then the plasma state is updated based on the estimators.

## Theory

Ordinarily, TARDIS is not concerned about the physical amount of time a packet spends traveling through the ejecta. Instead, we consider the "time of simulation" $\Delta t$ which is chosen to be the amount of time in which the photosphere emits the ensemble of packets (see [Energy Packet Initialization](../montecarlo/initialization.ipynb)). When looking at the estimators, a slightly different interpretation of the packets is necessary. Here, we view the packets as not carrying a discrete amount of energy $\varepsilon$ that is emitted in a time interval $\Delta t$, but as being a flow of energy that carries an energy $\varepsilon$ over a time $\Delta t$ -- that is, each packet is carrying a luminosity (energy per unit time) of $L = \frac{\varepsilon}{\Delta t}$. Now, we can say that if a packet spends a time $\delta t$ in the supernova's ejecta, it contributes an energy of $L\delta t= \frac{\varepsilon}{\Delta t}\delta t$ into the radiation energy of the ejecta.

To account for the effects of the Monte Carlo packets on the ejecta, TARDIS uses the packets to first determine the average radiation energy density $E$ throughout each shell, where the energy density is the total radiation energy in the shell divided by the volume of the shell ($\Delta V=4\pi (r_\mathrm{outer}^3-r_\mathrm{inner}^3)$). Therefore, we add up the amount of energy each packet contributes to the radiation energy in that shell, and divide by the total volume of the shell:
$$E=\frac{1}{\Delta V}\sum L_i\delta t_i=\frac{1}{\Delta V}\sum \frac{\varepsilon_i}{\Delta t}\delta t_i = \frac{1}{\Delta V\Delta t}\sum \varepsilon_i\delta t_i$$
where we sum over every Monte Carlo packet in the shell. Note that we are interested in the energy density in the co-moving frame (i.e. the energy density "according to the plasma," see [reference frames](../montecarlo/propagation.rst#reference-frames)) Now, we note that the amount of time the Monte Carlo packet spends in a shell is $\delta t = \frac{l_i}{c}$ where $l$ is the distance that the packet travels through the shell. Thus, our estimator is
$$E=\frac{1}{\Delta V\Delta t}\sum \varepsilon_i\frac{l_i}{c} = \frac{1}{c\Delta V\Delta t}\sum \varepsilon_i l_i.$$

Using this energy density, we can then calculate the mean radiation intensity in that shell using the relation $J=\frac{c}{4\pi} E$, which gives us
$$J=\frac{1}{4\pi\Delta V\Delta t}\sum \varepsilon_i l_i.$$
Since along any path the co-moving energy of the packet is continuously doppler shifted, we approximate this estimator using the co-moving energy at the midpoint of the packet's path.

Next, we calculate the mean radiation frequency in each shell. To do this we use the same procedure as with $J$, but in the sum we add up the frequency of each packet weighted by the amount of energy they contribute to the shell (once again equal to $\frac{\varepsilon_i l_i}{c\Delta t}$). Once again dividing by the shell's volume and multiplying by $\frac{c}{4\pi}$ (so we are actually weighting by the intensity contributed as opposed to just energy), we get
$$\bar \nu = \frac{1}{4\pi \Delta V \Delta t}\sum_i \varepsilon_i \nu_i l_i$$
where once again the co-moving energy and frequency of each packet is taken at the midpoint of the packet's path.

<div class="alert alert-info">
    
Note

Both estimators take on a different value in each shell.

</div>

<div class="alert alert-info">
    
Note

If the full effects of relativity are considered, the lengths (which are given in the lab frame) must be multiplied by $\sqrt{1-\frac{\mu^2 v^2}{c^2}}$ to account for length contraction (as the lengths used in the estimators must be in the co-moving frame).

</div>

These estimators allow us to calculate the [radiative temperature](../setup/model.ipynb#Radiative-Temperature) $T_\mathrm{rad}$ and [dilution factor](../setup/model.ipynb#Dilution-Factor) $W$ in each shell using

$$T_{\mathrm{rad}} = \frac{h}{k_{\mathrm{B}}} \frac{\pi^4}{360 \zeta(5)} \frac{\bar \nu}{J}$$

and

$$W = \frac{\pi J}{\sigma_{\mathrm{R}} T_{\mathrm{R}}^4}$$

where $h$ is Planck's constant, $k_{\mathrm{B}}$ is Boltzmann's constant, $\sigma_{\mathrm{R}}$ is the Stefan–Boltzmann constant, and $\zeta$ is the Riemann zeta function.

The new $T_\mathrm{rad}$ and $W$ are then used as inputs to the updated [plasma calculations](../setup/plasma/index.rst) which account for the effect of the monte carlo packets on the plasma state.

## Line Estimators

## Implementation

As previously discussed, a major component of each Monte Carlo iteration is the packet propagation process. During the packet propagation process this step, the $J$ and $\bar \nu$ estimators are updates every time a packet is moved to the next event location. Specifically, every time a packet is moved, $\varepsilon l$ is added to the "running total" $J$ estimator in the shell where the packet is, and $\varepsilon \nu l$ is added to the "running total" $\bar\nu$ estimator in the shell where the packet is (where $l$ is the distance the packet is moved, and $\varepsilon$ and $\nu$ are respectively the packet's co-moving energy and frequency at the center of the packet's path). The factor of $\frac{1}{4\pi\Delta V\Delta t}$ is multiplied to both estimators in each shell at the end of the iteration (including these factors only once all together at the end is for computational ease).

After each Monte Carlo iteration, the `advance_state()` method is called on the `Simulation` object. The estimators are then used to update the radiative temperature and dilution factor according to the [convergence strategy](convergence.rst), and the plasma state is recalculated (see [plasma calculations](../setup/plasma/index.rst)) using the updated radiative temperature and dilution factor as inputs. This process repeats until the final iteration or until convergence has been reached (see [convergence](convergence.rst#Convergence-Criteria)).

Similarly, during the propagation process, every time a packet passes through a Sobolev point, meaning it comes in resonance with an atomic line, the $J^b_{lu}$ for that atomic transition in the shell is incremented by $\frac{\varepsilon}{\nu_{lu}} D$, where $\varepsilon$ is the packet's energy, $\nu_{lu}$ is the transition's frequency, and $D$ is the doppler factor. As before, for computational ease, the factor $\frac{1}{4\pi \Delta V \Delta t} \frac{t_{\mathrm{exp}}}{c}$ is multiplied at the end.

If set to detailed mode (see [plasma configuration](../../io/config/components/plasma.rst)), the `J_blues` plasma property will will be replaced with the value of the $J^b_{lu}$ estimator. Otherwise, the `J_blues` in the plasma are calculated as they typically are in the plasma calculations, and the $J^b_{lu}$ estimator is only used for the [formal integral](../spectrum/sourceintegration.rst).

## Code Example

We now show a detailed example of how the plasma is updated using the estimators after a Monte Carlo iteration. First, we import the needed packages and set up a simulation (see [Setting Up the Simulation](../setup/index.rst)):

In [ ]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.io.atom_data.util import download_atom_data
import numpy as np

# We download the atomic data needed to run this notebook
download_atom_data('kurucz_cd23_chianti_H_He')

In [ ]:
tardis_config = Configuration.from_yaml('tardis_example.yml')
sim = Simulation.from_config(tardis_config)

model = sim.model
plasma = sim.plasma
runner = sim.runner

We show the initial radiative temperature, dilution factor, electron densities, and tau sobolevs in each radial cell:

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.get_value('electron_densities')

In [ ]:
plasma.get_value('tau_sobolevs')

We set the number of packets and we run one iteration of the Monte Carlo simulation:

In [ ]:
N_packets = 10000

# Using the commented out code below, we can also get the number of packets
# from the configuration -- try it out:
#N_packets = tardis_config.no_of_packets

sim.iterate(N_packets)

We now show the values of the three estimators previously mentioned:

In [ ]:
runner.j_estimator

In [ ]:
runner.nu_bar_estimator

In [ ]:
# Because most rows of the j_blue estimatior are partially or mostly
# zero, we show just rows with all nonzero elements
runner.j_blue_estimator[np.all(runner.j_blue_estimator != 0, axis=1)]

We note that the shape of the j_blue estimator and the tau_sobolevs are the same: namely, each contain a value for each possible atomic line transition in each radial cell (as opposed to the other two estimators which just have one value for each cell):

In [ ]:
plasma.get_value('tau_sobolevs').shape, runner.j_blue_estimator.shape

We now advance the state of the simulation based on the estimators, and demonstrate this by showing the four quantities we showed before running the simulation. Compare them with their values above!

In [ ]:
# When advance_state is called, a brief summary of the updated t_rad's
# and w's is given
sim.advance_state();

In [ ]:
model.t_rad

In [ ]:
model.w

In [ ]:
plasma.get_value('electron_densities')

In [ ]:
plasma.get_value('tau_sobolevs')